# PyGMTSAR Python Notebook for example S1A_Stack_CPGF_T173

## Alexey Pechnikov, Sep, 2021, https://github.com/mobigroup/gmtsar

## I'm a freelancer and that's my free time project. If you find it useful you are able to sponsor the project <a href="https://www.patreon.com/bePatron?u=54500608" data-patreon-widget-type="become-patron-button">Become a Patron!</a><script async src="https://c6.patreon.com/becomePatronButton.bundle.js"></script> on Patreon or order additional research, development and support on <a href="https://www.upwork.com/freelancers/~01e65e8e7221758623">Upwork</a>

## Download and unpack the example, create processing directory

## TODO

In [ ]:
import sys
import os

In [ ]:
import xarray as xr
import numpy as np
#import dask
#from dask.diagnostics import ProgressBar
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')
import glob
from io import StringIO
import xmltodict

In [ ]:
# to show PDF file
from wand.image import Image as WImage
# plotting modules
import hvplot.xarray  # noqa
import holoviews as hv
from holoviews import opts
from bokeh.models import FixedTicker
hv.extension('bokeh', 'matplotlib')
pd.options.plotting.backend = 'holoviews'

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# define common plot parameters
plot_opts = {'rasterize': True, 'xlabel':'Range', 'ylabel':'Azimuth', 'width':500, 'height':400}

## Custom modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from intf_ra2ll import intf_ra2ll_matrix, intf_ra2ll
from trans_ra2ll import trans_ra2ll_matrix
from PRM import PRM
from SBAS import SBAS

## Helper functions

In [ ]:
gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

In [ ]:
# stack by directory name for xr.open_mfdataset
def preprocess_dirname(ds):
    stack = ds.encoding["source"].split('/')[-2]
    return ds.assign(stack=stack)

def open_grids_stack(filenames):
    return xr.open_mfdataset(filenames, concat_dim='stack',combine='nested',
                               parallel=True, preprocess=preprocess_dirname)['z']

## Define parameters

In [ ]:
MASTER       = '2015-04-03'
WORKDIR      = 'raw'
DATADIR      = 'raw_orig'
DEMFILE      = 'topo/dem.grd'
#BASEDAYS     = 75
#BASEMETERS   = 75
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, WORKDIR).set_master(MASTER)
sbas.to_dataframe()

In [ ]:
title = 'Sentinel1 Frame on DEM plus GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem()[::4,::4].load().hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

In [ ]:
title = 'Sentinel1 Frame DEM cropped using GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem(geoloc=True).load()[::4,::4].hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

## Stack Images (for a single subswath only)

In [ ]:
sbas.stack_parallel()

## TODO: Baseline Table

## TODO: SBAS Baseline

## DEM in Radar Coordinates

In [ ]:
%%time

sbas.topo_ra()

In [ ]:
xr.open_dataarray(f'{WORKDIR}/topo_ra.grd').load().hvplot(cmap='kbc', title='Topo_ra', **plot_opts)

## Interferograms

In [ ]:
%%time

# we are able to define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing required)
sbas.intf_parallel([['2015-01-21', '2015-03-10']], wavelength=400, func=decimator)

## Plot results

In [ ]:
basename = f'{WORKDIR}/20150121_20150310_'

In [ ]:
corr = xr.open_dataarray(basename + 'corr.grd').load()
corr.hvplot(cmap='gray', clim=(0,0.8),  title='Correlation', **plot_opts)

In [ ]:
ticks = np.linspace(-np.pi, np.pi, 5).round(2)
color_ticks = FixedTicker(ticks=ticks)
xr.open_dataarray(basename + 'phase.grd').load()\
    .hvplot(clim=(-np.pi,np.pi), cmap='gist_rainbow_r', title='Phase [rad]', **plot_opts) + \
xr.open_dataarray(basename + 'phasefilt.grd').load()\
    .hvplot(clim=(-np.pi,np.pi), cmap='gist_rainbow_r', title='Phasefilt [rad]', **plot_opts)

## Unwrapping

In [ ]:
# we can generate and use a custom snaphu config file:
# conf = self.PRM().snaphu_config(defomax=0)
# we can use sbas.unwrap for a not parallel processing
sbas.unwrap_parallel([['2015-01-21', '2015-03-10']], threshold=CORRLIMIT)

In [ ]:
unwrap = xr.open_dataarray(basename + 'unwrap.grd').load()
unwrap.hvplot(cmap='jet', clim=tuple(np.nanquantile(unwrap, [0.01, 0.99])), title='Unwrap', **plot_opts)

## Bonus: Mask and interpolate low-coherence areas

In [ ]:
unwrap_masked = xr.where(corr<CORRLIMIT, np.nan, unwrap)
unwrap_masked_interp = sbas.nearest_grid(unwrap_masked)

unwrap_masked.to_netcdf(basename + 'unwrap_masked.nc')
unwrap_masked_interp.to_netcdf(basename + 'unwrap_masked_interp.nc')

unwrap_masked.hvplot(cmap='jet', clim=tuple(np.nanquantile(unwrap_masked, [0.01, 0.99])), title='Unwrap Masked', **plot_opts) + \
unwrap_masked_interp.hvplot(cmap='jet', clim=tuple(np.nanquantile(unwrap_masked_interp, [0.01, 0.99])), title='Unwrap Masked Interpolated', **plot_opts)